#### CSCE 670 :: Information Storage & Retrieval :: Texas A&M University :: Spring 2023


# Homework 2: Implicit Recommender Systems

### 100 points [9% of your final grade]

### Due: March 9 (Thursday) by 11:59pm

*Goals of this homework:* The objective of this homework is to get you familiar with the pipelines of implicit recommendation, turn the theories of collaborative filtering, matrix factorization, and bayesian personalized ranking (BPR) into practice, and compare the results with the naive non-personalized recommendation to see how personalized collaborative filtering algorithms, matrix factorization model, and the BPR model provide better recommendations for the ranking task (with implicit feedback).

*Submission instructions (eCampus):* To submit your homework, rename this notebook as `UIN_hw1.ipynb`. For example, my homework submission would be something like `555001234_hw1.ipynb`. Submit this notebook via eCampus (look for the homework 2 assignment there). Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit.

*Late submission policy:* For this homework, you may use as many late days as you like (up to the 5 total allotted to you).

*Collaboration policy:* You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Piazza, search StackOverflow, etc. But if you do get help in this way, you must inform us by **filling out the Collaboration Declarations at the bottom of this notebook**. 

*Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.*

The basic rule is that no student should explicitly share a solution with another student (and thereby circumvent the basic learning process), but it is okay to share general approaches, directions, and so on. If you feel like you have an issue that needs clarification, feel free to contact either me or the TA.

# Part 1. Recommendations with Implicit Feedback (20 points)

For this part, we're going to build a simple **non-personalized** implicit recommendation algorithm. Since feedback like user clicks, purchases, and views is much more widespread than explicit ratings, implicit recommenders offer great opportunities for far-reaching impact. 

Concretely, the task of implicit recommendation is to recommend items to users based on implicit signals from users, i.e., we only know what items a user is interested in, but have no idea what items the user dislikes. So for this case, the dataset we could use for this implicit recommendation experiment only contains binary data with 1 representing that the user likes the item, and with 0 representing that we don't know the user's preference towards the item. Because of this, we cannot use the same evaluation method as explicit recommendation. Instead, we need to evaluate the implicit recommendation quality by a ranking task.

For this part, we will:
* load and process the MovieLens 1M dataset,
* transfer the explicit dataset to implicit one,
* build a non-personalized implicit recommender, 
* and evaluate your recommender.

To start out, we need to prepare the data. We will use the MovieLens 1M data from https://grouplens.org/datasets/movielens/1m/ in this homework. Lucky for you, we are providing the file containing the ratings -- ratings.dat  -- so all you need to do is load the ratings.dat file in the notebook as a DataFrame variable using the Pandas library. The code to do this has been provided in the next cell, but you need to run it. The resulting data variables are: train_mat is the numpy array variable for training data of size (#users, #items) with non-zero entries representing user-item ratings, and zero entries representing unknown user-item ratings; and test_mat is the numpy array variable for testing data of size (#users, #items).

In [8]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

# set the seed value
np.random.seed(0)

data_df = pd.read_csv('./ratings.dat', sep='::', names=["UserID", "MovieID", "Rating", "Timestamp"], engine='python')

# First, generate dictionaries for mapping old id to new id for users and movies
unique_MovieID = data_df['MovieID'].unique()
unique_UserID = data_df['UserID'].unique()
j = 0
user_old2new_id_dict = dict()
for u in unique_UserID:
    user_old2new_id_dict[u] = j
    j += 1
j = 0
movie_old2new_id_dict = dict()
for i in unique_MovieID:
    movie_old2new_id_dict[i] = j
    j += 1
    
# Then, use the generated dictionaries to reindex UserID and MovieID in the data_df
user_list = data_df['UserID'].values
movie_list = data_df['MovieID'].values
for j in range(len(data_df)):
    user_list[j] = user_old2new_id_dict[user_list[j]]
    movie_list[j] = movie_old2new_id_dict[movie_list[j]]
data_df['UserID'] = user_list
data_df['movieID'] = movie_list

# generate train_df with 70% samples and test_df with 30% samples, and there should have no overlap between them.
train_index = np.random.random(len(data_df)) <= 0.7
train_df = data_df[train_index]
test_df = data_df[~train_index]

# generate train_mat and test_mat
num_user = len(data_df['UserID'].unique())
num_movie = len(data_df['MovieID'].unique())

train_mat = coo_matrix((train_df['Rating'].values, (train_df['UserID'].values, train_df['MovieID'].values)), shape=(num_user, num_movie)).astype(float).toarray()
test_mat = coo_matrix((test_df['Rating'].values, (test_df['UserID'].values, test_df['MovieID'].values)), shape=(num_user, num_movie)).astype(float).toarray()

It is very easy to transfer the explicit datasets you already generated to implicit ones: here, you will consider the watching behavior as the implicit signal showing that the user is interested in a movie. Thus, you can use the same train_df and test_df for implicit recommendation experiment with 'Rating' column ignored. And for train_mat and test_mat, you need to make all ratings to be value 1 and keep 0 entries the same.

In [9]:
# turn the explicit ratings to implicit feedback data
train_mat = (train_mat > 0).astype(float)
test_mat = (test_mat > 0).astype(float)

## Part 1a: Build the non-personalized recommender (10 points)

In this part, you need to build a non-personalized recommendation model to provide a ranked list of 50 movies as the recommendation for each user. The model is very simple: for each user, the recommendation list is to rank the unwatched movies by their **popularity**, where the popularity is the number of implicit feedback each movie gets. In this case, although it is non-personalized recommender, the recommendation results may be different for users because the unwatched movies are different across users.

In the next cell, write your code to generate the ranked lists of movies by the popularity based recommendation algorithm for every user, store the result in a numpy array of size (#user, 50), entry (u, k) represents the movie id that is ranked at position k in the recommendation list to user u. Print out the top-5 recommended movies and their popularity for the first user (with id 0).

Hint: the popularity can only be calculated from train_mat, you cannot use test_mat here. 

In [87]:
# # Generate a ranked list of movies by the popularity based recommendation algorithm. And print out the id and popularity of the top5 movies for the first user.
# # Your Code Here...

popularity = train_mat.sum(axis=0)

sorted_movies = np.argsort(popularity)[: : -1]

num_users, num_items = train_mat.shape
np_recommendations = np.zeros((num_users, 50), dtype=int)
for u in range(num_users):
    unwatched_movies = np.where(train_mat[u, :] == 0)[0]
    sorted_unwatched_movies = sorted_movies[np.isin(sorted_movies, unwatched_movies)]
    np_recommendations[u, :] = sorted_unwatched_movies[:50]

top_5_popularity = popularity[top_5_movies]
top_5_movies = np_recommendations[0, :5]
print("Top-5 for first user:")
for i in range(0,5):
    print(f"Movie: {top_5_movies[i]}; popularity score: {top_5_popularity[i]}")

Top-5 for first user:
Movie: 104; popularity score: 2423.0
Movie: 124; popularity score: 2086.0
Movie: 64; popularity score: 2064.0
Movie: 113; popularity score: 1862.0
Movie: 97; popularity score: 1845.0


In [40]:
np_recommendations

array([[104, 124,  64, ..., 206, 315,  58],
       [ 44,  97,  22, ...,  27, 168,  19],
       [ 97,  48, 132, ...,  58, 280,  27],
       ...,
       [104,  44, 124, ..., 210,  78, 213],
       [104, 124, 113, ...,  67, 547, 206],
       [ 44,  64, 113, ..., 538, 145,  33]])

## Part 1b: Evaluate the non-personalized recommender (10 points)

In this part, you need to evaluate your non-personalized recommendation by the held-out testing dataset test_mat for each user. For the implicit recommendation, two typical metrics are recall@k and precision@k. Here, you need to write the code to calculate recall@k and  precision@k for k=5, 20, 50 for each user, i.e., six metrics for every user. And please print out the average over all users for these six metrics.

Hint: if a user does not have any testing samples in test_mat, do not include this user in the final averaged metric.


In [53]:
# Calculate recall@k and precision@k with k=5, 20, 50 and print out the average over all users for these 9 metrics.
# Your Code Here...
recommendation = np_recommendations

recalls = np.zeros(3)
precisions = np.zeros(3)
user_count = 0.

user_test_like = []
for u in range(num_user):
    user_test_like.append(np.where(test_mat[u, :] > 0)[0])

for u in range(num_user):
        
    test_like = user_test_like[u]
    test_like_num = len(test_like)
    if test_like_num == 0:
        continue
    rec = recommendation[u, :]
    hits = np.zeros(3)
    for k in range(50):
        if rec[k] in test_like:
            if k < 50:
                hits[2] += 1
                if k < 20:
                    hits[1] += 1
                    if k < 5:
                        hits[0] += 1
    recalls[0] += (hits[0] / test_like_num)
    recalls[1] += (hits[1] / test_like_num)
    recalls[2] += (hits[2] / test_like_num)
    precisions[0] += (hits[0] / 5.)
    precisions[1] += (hits[1] / 20.)
    precisions[2] += (hits[2] / 50.)
    user_count += 1

recalls /= user_count
precisions /= user_count

print('recall@5\t[%.6f],\t||\t recall@20\t[%.6f],\t||\t recall@50\t[%.6f]' % (recalls[0], recalls[1], recalls[2]))
print('precision@5\t[%.6f],\t||\t precision@20\t[%.6f],\t||\t precision@50\t[%.6f]' % (precisions[0], precisions[1], precisions[2]))
print('')


recall@5	[0.037859],	||	 recall@20	[0.108581],	||	 recall@50	[0.193603]
precision@5	[0.272980],	||	 precision@20	[0.210364],	||	 precision@50	[0.157795]



In [54]:
rec_np = recalls
pre_np = precisions

# Part 2. Build the Collaborative Filtering Model for Implicit Feedback (25 points)

In this part, we will need to build **personalized** models instead of non-personalized models as in Part 1. We study how collaborative filtering algorithms work for recommendations with implicit feedback. The overall pipeline is the same as in Part 1. But now you need to implement a user-user collaborative filtering algorithm for recommendation. You will also evaluate your personalized models to compare them with the non-personalized one in Part 1.

In this part, you will use the same MovieLens 1M dataset, and:

* write the code to implement a user-user collaborative filtering algorithm,
* and evaluate your recommender.

First, we need to load and preprocess the experiment dataset. We still use the MovieLens 1M data from https://grouplens.org/datasets/movielens/1m/ in this part.

Then, you need to write code to implement a user-user collaborative filtering algorithm with **cosine similarity**. Although we have not discussed in class how to use collaborative filtering for implicit feedback, it is quite straightforward. 

* We first need to calculate the cosine similarity between users based on the binary feedback vectors of users; 
* Then, for a specific user $u$, we predict a preference vector for the user $u$ by weighted averaging the binary feedback vectors of N users who are most similar to $u$;
* And last, rank the movies based on the predicted preference vector of the user $u$ as recommendations. 

The predicted preference score from user $u$ to movie $i$ can be calculated as: $p_{u,i}=\frac{\sum_{u^\prime\in N}s(u,u^\prime)r_{u^\prime,i}}{\sum_{u^\prime\in N}|s(u,u^\prime)|}$, where $s(u,u^\prime)$ is the cosine similarity, and we set the size of $N$ as 10.

In the next cell, write your code to generate the ranked lists of movies by this user-user collaborative filtering recommendation algorithm for every user, store the result in a numpy array of size (#user, 50), where entry (u, k) represents the movie id that is ranked at position k in the recommendation list to user u. 

* Hint: for a user $u$, the movies user $u$ liked in the train_mat should be excluded in the top 50 recommendation list

In [33]:
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

# calculate cosine similarity between users based on binary feedback
user_sim = cosine_similarity(train_mat)

# set the number of similar users to consider for each user
N = 10

# array to store cos recommend
cos_recommend = np.zeros((num_users, 50), dtype=int)

#now we need to take top 10 similar users for each user and recommend movies for this user
for u in range(num_users):
    u_sim = user_sim[u]
    u_sim[u] = -np.inf
    top_10_uids = np.argsort(u_sim)[::-1][:N]
    top_10_usim = u_sim[top_10_uids]
    
    denominator = sum(top_10_usim)

    
    # Find the movies that the user has not watched
    unwatched_movies = np.where(train_mat[u, :] == 0)[0]
    
    movie_pref_u_unwatched = np.zeros(len(unwatched_movies))
    #iterator k
    k = 0
    
    #now iterate movies and find p for each
    for m in unwatched_movies:
        # Extract the m-th column for the rows in top_10_uids
        sim_users_rating = train_mat[top_10_uids, m]
        #dot between top_10_usim and sim_users_rating
        numerator = np.dot(sim_users_rating, top_10_usim)
        movie_pref_u_unwatched[k] = numerator/denominator
        k += 1
        
    #sort these unwatched movies by their pref score
    sort_uwatched_pref_ind = np.argsort(movie_pref_u_unwatched)[::-1]
    
    #store the movie id's of these 
    rec_movies = unwatched_movies[sort_uwatched_pref_ind]
        
    # Store the IDs of the top 50 recommended movies for user u
    cos_recommend[u] = rec_movies[:50]
cos_recommend

array([[ 207,   10,  390, ..., 1314,  389,  733],
       [  92,  171,  189, ..., 1228,  719,  217],
       [  97,   40,  381, ...,  643,  171,  644],
       ...,
       [  44,  486,  104, ..., 1297,  184,  623],
       [1465,  726, 1623, ..., 1106, 2313, 1633],
       [ 499,  136,    0, ..., 2066, 1529, 1528]])

Last, you need to evaluate your user-user collaborative filtering algorithm by the held-out testing dataset test_mat for each user. Here, we use the same metrics recall@k and precision@k we used in Part 2c of Module 1. So you can use the same code here to calculate recall@k and precision@k for k=5, 20, 50 for each user, i.e., six metrics for every user. And please print out the average over all users for these six metrics.

In [55]:
# Calculate recall@k, precision@k with k=5, 20, 50 and print out the average over all users for these 6 metrics.
# Your Code Here...

recommendation = cos_recommend

recalls = np.zeros(3)
precisions = np.zeros(3)
user_count = 0.

user_test_like = []
for u in range(num_user):
    user_test_like.append(np.where(test_mat[u, :] > 0)[0])

for u in range(num_user):
        
    test_like = user_test_like[u]
    test_like_num = len(test_like)
    if test_like_num == 0:
        continue
    rec = recommendation[u, :]
    hits = np.zeros(3)
    for k in range(50):
        if rec[k] in test_like:
            if k < 50:
                hits[2] += 1
                if k < 20:
                    hits[1] += 1
                    if k < 5:
                        hits[0] += 1
    recalls[0] += (hits[0] / test_like_num)
    recalls[1] += (hits[1] / test_like_num)
    recalls[2] += (hits[2] / test_like_num)
    precisions[0] += (hits[0] / 5.)
    precisions[1] += (hits[1] / 20.)
    precisions[2] += (hits[2] / 50.)
    user_count += 1

recalls /= user_count
precisions /= user_count

print('recall@5\t[%.6f],\t||\t recall@20\t[%.6f],\t||\t recall@50\t[%.6f]' % (recalls[0], recalls[1], recalls[2]))
print('precision@5\t[%.6f],\t||\t precision@20\t[%.6f],\t||\t precision@50\t[%.6f]' % (precisions[0], precisions[1], precisions[2]))
print('')

recall@5	[0.070497],	||	 recall@20	[0.191733],	||	 recall@50	[0.321965]
precision@5	[0.378146],	||	 precision@20	[0.290911],	||	 precision@50	[0.221732]



In [56]:
rec_cf = recalls
pre_cf = precisions

**Question (5 points):** do you observe a better result compared with models implemented in the previous part? What's reason do you think that brings this improvement?

# Part 3. Build the Matrix Factorization Model for Implicit Feedback (25 points)

Now we turn to study how matrix factorization algorithms works for recommendations with implicit feedback. The overall pipeline is the same as in Part 1 and Part 2. But now you need to implement a matrix factorization algorithm for recommendation. You will also compare your matrix factorization model to the non-personalized one in Part 1 and collaborative filtering one in Part 2.

In this part, you will use the same MovieLens 1M dataset, and:
* write the code to implement a matrix factorization algorithm,
* and evaluate your recommender.

First, let's recap the matrix factorization model introduced in class. The MF model can be mathematically represented as: 

<center>$\underset{\mathbf{P},\mathbf{Q}}{\text{min}}\,\,L=\sum_{(u,i)\in\mathcal{O}}(\mathbf{P}_u\cdot\mathbf{Q}^\top_i-r_{u,i})^2+\lambda(\lVert\mathbf{P}\rVert^2_{\text{F}}+\lVert\mathbf{Q}\rVert^2_{\text{F}})$,</center>
    
where $\mathbf{P}$ is the user latent factor matrix of size (#user, #latent); $\mathbf{Q}$ is the movie latent factor matrix of size (#movie, #latent); $\mathcal{O}$ is a user-movie pair set containing all user-movie pairs having ratings in train_mat; $r_{u,i}$ represents the rating for user u and movie i; $\lambda(\lVert\mathbf{P}\rVert^2_{\text{F}}+\lVert\mathbf{Q}\rVert^2_{\text{F}})$ is the regularization term to overcome overfitting problem, $\lambda$ is the regularization weight (a hyper-parameter manually set by developer, i.e., you), and $\lVert\mathbf{P}\rVert^2_{\text{F}}=\sum_{x}\sum_{y}(\mathbf{P}_{x,y})^2$, $\lVert\mathbf{Q}\rVert^2_{\text{F}}=\sum_{x}\sum_{y}(\mathbf{Q}_{x,y})^2$. Such an L function is called the **loss function** for the matrix factorization model. The goal of training an MF model is to find appropriate $\mathbf{P}$ and $\mathbf{Q}$ to minimize the loss L.

Although we have not discussed in class how to use matrix factorization algorithm for implicit feedback, it is quite straightforward. The main idea is the same as the MF model for explicit ratings. But instead of predicting explicit ratings, here the MF is to predict binary ratings based on which movies are ranked for users.

The only challenge now is that in an implicit feedback dataset, there is only positive signal (i.e., '1' in the train_mat) without negative signal. Hence, to let our MF work for this implicit feedback, a simple but powerful method -- random negative sampling -- is adopted. The main idea is that in each training epoch, we randomly sample user-movie pairs without positive feedback in train_mat (user-movie pairs with '0' in train_mat) to be negative feedback, and mix them with positive feedback as the training data to train the MF model. The **negative sampling method is already provided**.

In the Next cell, you need to complete the MF_implicit class. There are five functions in this class: 

* 'init' (**provided**) is to initialize the variables the MF class needs, which takes 5 inputs: train_mat, test_mat, latent, lr, and reg. 'train_mat' and 'test_mat' are the corresponfing training and testing matrices we have. 'latent' represents the latent dimension we set for the MF model. 'lr' represents the learning rate, i.e., the update step in each optimization iteration, default is 0.01. 'reg' represents the regularization weight, i.e., the $\lambda$ in the MF formulation. 

* 'negative_sampling' (**provided**) is to do the random negative sampling to generate negative signals for training the MF. It returns a list of users and a list of movies as the training samples, mixing positive and negative user-movie pairs.

* 'test' (**provided**) is to evaluate the trained MF on test_mat and print out recall@k and precision@k. 

* 'predict' (**need to be completed**) is to generate the ranked lists of movies by the trained MF for every user, store the ranking result (named 'recommendation') in a numpy array of size (#user, 50), where entry (u, k) represents the movie id that is ranked at position k in the recommendation list to user u. Return the 'recommendation' variable. 

* 'train' (**need to be completed**) is to train the MF model. There is only one input to this function: an int variable 'epoch' to indicate how many epochs for training the model. The main logic of this function is the same as the one you already implemented in Part 1a. The main body of this function should be a loop for 'epoch' iterations. In each iteration, following the algorithm to update the MF model:

        1. Call 'negative_sampling()' to generate a list of users and a list of movies mixing positive and negative user-movie pairs as training samples.
        2. Randomly shuffle training user-movie pairs  (i.e., user-movie pairs from step 1)
        2. Have an inner loop to iterate each user-movie pair:
                a. given a user-movie pair (u,i), update the user latent factor and movie latent factor by gradient decsent:    
<center>$\mathbf{P}_u=\mathbf{P}_u-\gamma [2(\mathbf{P}_u\cdot\mathbf{Q}_i^\top-r_{u,i})\cdot\mathbf{Q}_i+2\lambda\mathbf{P}_u]$</center>    
<center>$\mathbf{Q}_i=\mathbf{Q}_i-\gamma [2(\mathbf{P}_u\cdot\mathbf{Q}_i^\top-r_{u,i})\cdot\mathbf{P}_u+2\lambda\mathbf{Q}_i]$</center>    
<center>where $\mathbf{P}_u$ and $\mathbf{Q}_i$ are row vectors of size (1, #latent), $\gamma$ is learning rate (default is 0.01), $\lambda$ is regularization weight, and $r_{u,i}$ now takes binary value.</center>
        
        3. After iterating over all user-movie pairs, call 'test()' to evaluate the current MF model.

**Note: you are not supposed to delete or modify the provided code.**

**Note: for this part, it is necessary to read and understand the provided code, because you will need to call the provided functions in your code.**

In [47]:
class MF_implicit:
    def __init__(self, train_mat, test_mat, latent=5, lr=0.01, reg=0.01):
        self.train_mat = train_mat  # the training rating matrix of size (#user, #movie)
        self.test_mat = test_mat  # the training rating matrix of size (#user, #movie)
        
        self.latent = latent  # the latent dimension
        self.lr = lr  # learning rate
        self.reg = reg  # regularization weight, i.e., the lambda in the objective function
        
        self.num_user, self.num_movie = train_mat.shape
        
        self.sample_user, self.sample_movie = self.train_mat.nonzero()  # get the user-movie paris having ratings in train_mat
        self.num_sample = len(self.sample_user)  # the number of user-movie pairs having ratings in train_mat

        self.user_test_like = []
        for u in range(self.num_user):
            self.user_test_like.append(np.where(self.test_mat[u, :] > 0)[0])

        self.P = np.random.random((self.num_user, self.latent))  # latent factors for users, size (#user, self.latent), randomly initialized
        self.Q = np.random.random((self.num_movie, self.latent))  # latent factors for users, size (#movie, self.latent), randomly initialized
        
    def negative_sampling(self):
        negative_movie = np.random.choice(np.arange(self.num_movie), size=(len(self.sample_user)), replace=True)
        true_negative = self.train_mat[self.sample_user, negative_movie] == 0
        negative_user = self.sample_user[true_negative]
        negative_movie = negative_movie[true_negative]
        return np.concatenate([self.sample_user, negative_user]), np.concatenate([self.sample_movie, negative_movie])

    def train(self, epoch=20):
        """
        Goal: Write your code to train your matrix factorization model for epoch iterations in this function
        Input: epoch -- the number of training epoch 
        """
        for ep in range(epoch):
            """ 
            Write your code here to implement the training process for one epoch, 
            at the end of each epoch, run self.test() to evaluate current version of MF.
            """
            user_list, movie_list = self.negative_sampling()
            
            #randomly shuffle
            
            # Combine the two lists using zip()
            combined_list = list(zip(user_list, movie_list))

            # Shuffle the combined list randomly
            random.shuffle(combined_list)

            # Unpack the shuffled list into separate user and movie lists using zip()
            shuffled_user_list, shuffled_movie_list = zip(*combined_list)
            
            
            
            # update latent matrices P and Q
            for u, i in zip(shuffled_user_list, shuffled_movie_list):
                err = 2 * (np.dot(self.P[u], self.Q[i].T) - (self.train_mat[u, i]))
                p_u = self.P[u].copy()
                q_i = self.Q[i].copy()
                self.P[u] = p_u - (self.lr * ((err * q_i) + (2 * self.reg * p_u)))
                self.Q[i] = q_i - (self.lr * ((err * p_u) + (2 * self.reg * q_i)))

            print("epoch")
            print(ep)
            pre, rec = self.test()
            
        return pre, rec            
        """
        End of your code for this function
        """
            
    def predict(self):
        """
        Write your code here to implement the prediction function, which generates the ranked lists of movies 
        by the trained MF for every user, store the result (named 'recommendation') in a numpy array of size (#user, 50), where entry (u, k) 
        represents the movie id that is ranked at position k in the recommendation list to user u. Return the 'recommendation' variable. 
        """
        recommendation = np.zeros((self.num_user, 50))
        for u in range(0,self.num_user):
            # predict the ranking score for unrated movies
            ranking = np.dot(self.P[u], self.Q.T)
            # exclude rated movies
            ranking[self.train_mat[u].nonzero()] = -np.inf
            # top 50 recommended movies
            recommendation[u] = np.argsort(ranking)[::-1][:50]
        return recommendation
    
    def test(self):
        recommendation = self.predict()

        recalls = np.zeros(3)
        precisions = np.zeros(3)
        user_count = 0.

        for u in range(self.num_user):
            test_like = self.user_test_like[u]
            test_like_num = len(test_like)
            if test_like_num == 0:
                continue
            rec = recommendation[u, :]
            hits = np.zeros(3)
            for k in range(50):
                if rec[k] in test_like:
                    if k < 50:
                        hits[2] += 1
                        if k < 20:
                            hits[1] += 1
                            if k < 5:
                                hits[0] += 1
            recalls[0] += (hits[0] / test_like_num)
            recalls[1] += (hits[1] / test_like_num)
            recalls[2] += (hits[2] / test_like_num)
            precisions[0] += (hits[0] / 5.)
            precisions[1] += (hits[1] / 20.)
            precisions[2] += (hits[2] / 50.)
            user_count += 1

        recalls /= user_count
        precisions /= user_count

        print('recall@5\t[%.6f],\t||\t recall@20\t[%.6f],\t||\t recall@50\t[%.6f]' % (recalls[0], recalls[1], recalls[2]))
        print('precision@5\t[%.6f],\t||\t precision@20\t[%.6f],\t||\t precision@50\t[%.6f]' % (precisions[0], precisions[1], precisions[2]))
        print('')
        
        return precisions, recalls

Now, run the next cell to build and train your implemented MF model for implicit feedback. The expected time used for training one epoch is 20s to 2 min.

In [48]:
mf_implicit = MF_implicit(train_mat, test_mat, latent=5, lr=0.01, reg=0.0001)
pre_mf,rec_mf = mf_implicit.train(epoch=20)

epoch
0
recall@5	[0.013178],	||	 recall@20	[0.056748],	||	 recall@50	[0.134744]
precision@5	[0.144172],	||	 precision@20	[0.142425],	||	 precision@50	[0.127778]

epoch
1
recall@5	[0.023075],	||	 recall@20	[0.082760],	||	 recall@50	[0.167991]
precision@5	[0.202914],	||	 precision@20	[0.178990],	||	 precision@50	[0.145361]

epoch
2
recall@5	[0.026367],	||	 recall@20	[0.087157],	||	 recall@50	[0.172315]
precision@5	[0.222086],	||	 precision@20	[0.184851],	||	 precision@50	[0.148738]

epoch
3
recall@5	[0.031098],	||	 recall@20	[0.094336],	||	 recall@50	[0.174677]
precision@5	[0.242086],	||	 precision@20	[0.191921],	||	 precision@50	[0.148487]

epoch
4
recall@5	[0.028618],	||	 recall@20	[0.092695],	||	 recall@50	[0.180527]
precision@5	[0.236457],	||	 precision@20	[0.195290],	||	 precision@50	[0.155699]

epoch
5
recall@5	[0.029047],	||	 recall@20	[0.090238],	||	 recall@50	[0.178050]
precision@5	[0.257616],	||	 precision@20	[0.202641],	||	 precision@50	[0.162689]

epoch
6
recall@5	[0.030731],

In [49]:
print('recall@5\t[%.6f],\t||\t recall@20\t[%.6f],\t||\t recall@50\t[%.6f]' % (rec_mf[0], rec_mf[1], rec_mf[2]))
print('precision@5\t[%.6f],\t||\t precision@20\t[%.6f],\t||\t precision@50\t[%.6f]' % (pre_mf[0], pre_mf[1], pre_mf[2]))
print('')

recall@5	[0.039313],	||	 recall@20	[0.125554],	||	 recall@50	[0.245815]
precision@5	[0.291060],	||	 precision@20	[0.245944],	||	 precision@50	[0.202526]



**Question (5 points):** do you observe a better result compared with models implemented in previous two parts? What's reason do you think that brings this improvement?

# Part 4: Build the Bayesian Personalized Ranking (BPR) Model with Implicit Feedback (20 points)

In this first part, you will need to build a **Bayesian Personalized Ranking (BPR)** model. You will also evaluate your BPR model to compare with the non-personalized one in Part 1, neighborhood-based collaborative filtering model in Part 2, and matrix factorization model in Part 3. 

For this part, we will:

* build a BPR model,
* and evaluate your recommender.

We still use the MovieLens 1M data from https://grouplens.org/datasets/movielens/1m/ in this part, and use the same loading and preprocessing process.

First, let's implement the Bayesian Personalized Ranking model introduced in class. The BPR model can be mathematically represented as: 

<center>$\underset{\mathbf{P},\mathbf{Q}}{\text{max}}\,\,L=\sum_{(u,i,j)\in\mathcal{O}}\sigma(\mathbf{P}_u\cdot\mathbf{Q}^\top_i-\mathbf{P}_u\cdot\mathbf{Q}^\top_j)+\lambda(\lVert\mathbf{P}\rVert^2_{\text{F}}+\lVert\mathbf{Q}\rVert^2_{\text{F}})$,</center>
    
where $\mathbf{P}$ is the user latent factor matrix of size (#user, #latent); $\mathbf{Q}$ is the movie latent factor matrix of size (#movie, #latent); $\sigma(\cdot)$ is the Sigmoid function; $\mathcal{O}$ is a (user, positive movie, negative movie) tuple set, and each tuple $(u,i,j)$ is a training sample with user $u$ and a posotive movie $i$ with value 1 in train_mat and a negative movie $j$ with value 0 in train_mat; $\lambda(\lVert\mathbf{P}\rVert^2_{\text{F}}+\lVert\mathbf{Q}\rVert^2_{\text{F}})$ is the regularization term to overcome overfitting problem, $\lambda$ is the regularization weight (a hyper-parameter manually set by the developer, i.e., you), and $\lVert\mathbf{P}\rVert^2_{\text{F}}=\sum_{x}\sum_{y}(\mathbf{P}_{x,y})^2$, $\lVert\mathbf{Q}\rVert^2_{\text{F}}=\sum_{x}\sum_{y}(\mathbf{Q}_{x,y})^2$. Such an L function is called the **loss function** for the matrix factorization model. The goal of training a BPR model is to find appropriate $\mathbf{P}$ and $\mathbf{Q}$ to **maximize** the loss L.

To implement such a BPR model, here we will write a Python class for the model. Similar to the implicit_MF model you have already implemented in Part 3, there are five functions in this BPR class: 

* The 'init' function (**provided**) is to initialize the variables the BPR class needs, which takes 5 inputs: train_mat, test_mat, latent, lr, and reg. 'train_mat' and 'test_mat' are the corresponfing training and testing matrices we have. 'latent' represents the latent dimension we set for the BPR model. 'lr' represents the learning rate, i.e., the update step in each optimization iteration, default is 0.01. 'reg' represents the regularization weight, i.e., the $\lambda$ in the BPR formulation.

* 'negative_sampling' (**provided**) is to do the random negative sampling to generate negative signals for training the BPR. It returns a list of users, a list of positive movies of these users, and a list of negative movies of these user. These three lists form the training set $\mathcal{O}$.

* 'test' (**provided**) is to evaluate the trained MF on test_mat and print out recall@k and precision@k. 

* 'predict' (**provided**) is to generates the ranked lists of movies by the trained MF for every user, store the ranking result (named 'recommendation') in a numpy array of size (#user, 50), where entry (u, k) represents the movie id that is ranked at position k in the recommendation list to user u. Return the 'recommendation' variable. 

* 'train' (**need to be completed**) is to train the BPR model. There is only one input to this function: an int variable 'epoch' to indicate how many epochs for training the model. The main process of one training epoch is:

        1. Call 'negative_sampling()' to generate training samples: a list of users, a list of positive movies of these users, and a list of negative movies of these users.
        2. Randomly shuffle training samples from step 1.
        3. Have an inner loop to iterate each (user, positive movie, negative movie) tuple in the shuffled training samples:
                a. given the current training tuple -- a user u, a positive movie i, and a negative movie j, update the user latent factor and movie latent factor by gradient decsent:
<center>$\mathbf{P}_u=\mathbf{P}_u - \gamma[-\frac{e^{-\widehat{x}}}{1 + e^{-\widehat{x}}} (\mathbf{Q}_i - \mathbf{Q}_j) + \lambda\mathbf{P}_u]$</center>
<center>$\mathbf{Q}_i=\mathbf{Q}_i - \gamma[-\frac{e^{-\widehat{x}}}{1 + e^{-\widehat{x}}} \mathbf{P}_U + \lambda\mathbf{Q}_i]$</center>
<center>$\mathbf{Q}_j=\mathbf{Q}_j - \gamma[\frac{e^{-\widehat{x}}}{1 + e^{-\widehat{x}}} \mathbf{P}_U + \lambda\mathbf{Q}_j]$</center>   
<center>where $\mathbf{P}_u$, $\mathbf{Q}_i$, and $\mathbf{Q}_j$ are of size (1, #latent), $\gamma$ is learning rate (default is 0.01), $\lambda$ is regularization weight, and $\widehat{x}=\mathbf{P}_u\cdot\mathbf{Q}^\top_i-\mathbf{P}_u\cdot\mathbf{Q}^\top_j$.</center>
        
        3. After iterate all training samples, call 'test()' to evaluate the current BPR model.


In the next cell, you will need to fill in the 'train' function based on the description above. 

*Hint that similar to the training process of MF in Part 3, in each update iteration, you need to use the old P and Q from last update iteration to update P and Q in current update iteration.

**NOTE that you should not delete or modify the provided code.**

In [51]:
class BPR:
    def __init__(self, train_mat, test_mat, latent=5, lr=0.01, reg=0.01):
        self.train_mat = train_mat  # the training rating matrix of size (#user, #movie)
        self.test_mat = test_mat  # the training rating matrix of size (#user, #movie)
        
        self.latent = latent  # the latent dimension
        self.lr = lr  # learning rate
        self.reg = reg  # regularization weight, i.e., the lambda in the objective function
        
        self.num_user, self.num_movie = train_mat.shape
        
        self.positive_user, self.positive_movie = self.train_mat.nonzero()  # get the user-movie paris having ratings in train_mat

        self.user_test_like = []
        for u in range(self.num_user):
            self.user_test_like.append(np.where(self.test_mat[u, :] > 0)[0])

        self.P = np.random.random((self.num_user, self.latent))  # latent factors for users, size (#user, self.latent), randomly initialized
        self.Q = np.random.random((self.num_movie, self.latent))  # latent factors for users, size (#movie, self.latent), randomly initialized
        
    def negative_sampling(self): 
        # do the negative sampling for each of the positive user-movie pair. Here we set negative sampling rate as 2, 
        # i.e., for each positive user-movie pair, randomly sample two negative movies. 
        # This function returns final training data: a user list, a positive movie list, and a negative movie list
        sample_user = np.tile(self.positive_user, 2)
        sample_pos_movie = np.tile(self.positive_movie, 2)
        sample_neg_movie = np.random.choice(np.arange(self.num_movie), size=(len(sample_user)), replace=True)
        true_negative = self.train_mat[sample_user, sample_neg_movie] == 0
        return sample_user[true_negative], sample_pos_movie[true_negative], sample_neg_movie[true_negative]

    def train(self, epoch=20):
        """
        Goal: Write your code to train your matrix factorization model for epoch iterations in this function
        Input: epoch -- the number of training epoch 
        """
        for ep in range(epoch):
            """ 
            Write your code here to implement the training process for one epoch, 
            at the end of each epoch, run self.test() to evaluate current version of BPR.
            """
            users, pos_items, neg_items = self.negative_sampling()
            samples = list(zip(users, pos_items, neg_items))
            np.random.shuffle(samples)
            # Unpack the shuffled list into separate user and movie lists using zip()
            shuffled_user_list, shuffled_movie_pos,shuffled_movie_neg = zip(*samples)
            for u, i, j in zip(shuffled_user_list, shuffled_movie_pos,shuffled_movie_neg):
                p_u = self.P[u].copy()
                q_i = self.Q[i].copy()
                q_j = self.Q[j].copy()

#                 x = np.dot(self.P[u], self.Q[i] - self.Q[j])
                x = np.dot(self.P[u], self.Q[i].T) - np.dot(self.P[u], self.Q[j].T)
                err = np.exp(-x) / (1.0 + np.exp(-x))
                self.P[u] = p_u - ( self.lr * ( (-1 * err * (q_i - q_j)) + ( self.reg * p_u)  )   )
                self.Q[i] = q_i - ( self.lr * ( (-1 * err * (p_u)) + ( self.reg * q_i)  )   )
                self.Q[j] = q_j - ( self.lr * ( (err * (p_u)) + ( self.reg * q_j)  )   )
            
            print("epoch")
            print(ep)
            
            pre, rec = self.test()
            
        return pre, rec
        """
        End of your code for this function
        """
            
    def predict(self):
        # The prediction function, which generates the ranked lists of movies 
        # by the trained BPR for every user, store the result (named 'recommendation') in a numpy array of size (#user, 50), where entry (u, k) 
        # represents the movie id that is ranked at position k in the recommendation list to user u. Return the 'recommendation' variable. 
        prediction_mat = np.matmul(self.P, self.Q.T)
        recommendation = []
        for u in range(self.num_user):
            scores = prediction_mat[u]
            train_like = np.where(train_mat[u, :] > 0)[0]
            scores[train_like] = -9999
            top50_iid = np.argpartition(scores, -50)[-50:]
            top50_iid = top50_iid[np.argsort(scores[top50_iid])[-1::-1]]
            recommendation.append(top50_iid)
        recommendation = np.array(recommendation)
        return recommendation
    
    def test(self):
        recommendation = self.predict()

        recalls = np.zeros(3)
        precisions = np.zeros(3)
        user_count = 0.

        for u in range(self.num_user):
            test_like = self.user_test_like[u]
            test_like_num = len(test_like)
            if test_like_num == 0:
                continue
            rec = recommendation[u, :]
            hits = np.zeros(3)
            for k in range(50):
                if rec[k] in test_like:
                    if k < 50:
                        hits[2] += 1
                        if k < 20:
                            hits[1] += 1
                            if k < 5:
                                hits[0] += 1
            recalls[0] += (hits[0] / test_like_num)
            recalls[1] += (hits[1] / test_like_num)
            recalls[2] += (hits[2] / test_like_num)
            precisions[0] += (hits[0] / 5.)
            precisions[1] += (hits[1] / 20.)
            precisions[2] += (hits[2] / 50.)
            user_count += 1

        recalls /= user_count
        precisions /= user_count

        print('recall@5\t[%.6f],\t||\t recall@20\t[%.6f],\t||\t recall@50\t[%.6f]' % (recalls[0], recalls[1], recalls[2]))
        print('precision@5\t[%.6f],\t||\t precision@20\t[%.6f],\t||\t precision@50\t[%.6f]' % (precisions[0], precisions[1], precisions[2]))
        print('')
        
        return precisions, recalls
    
    

Now, let's train a BPR model based on your implementation. The code is provided, you just need to execute the next cell. The expectations are: 

* first, the code can be successfully executed without error; 
* and second, the recall@k and precision@k on **test_mat** of each training epoch should be printed out for all 20 epochs.


* Hint: the expected time used for training is around 1 minute to 5 minutes per training epoch.

In [52]:
bpr = BPR(train_mat, test_mat, latent=5, lr=0.01, reg=0.001)
pre_bpr,rec_bpr = bpr.train(epoch=20)

epoch
0
recall@5	[0.033957],	||	 recall@20	[0.103853],	||	 recall@50	[0.188034]
precision@5	[0.260993],	||	 precision@20	[0.205911],	||	 precision@50	[0.155434]

epoch
1
recall@5	[0.033963],	||	 recall@20	[0.105015],	||	 recall@50	[0.187602]
precision@5	[0.260265],	||	 precision@20	[0.207094],	||	 precision@50	[0.154679]

epoch
2
recall@5	[0.035242],	||	 recall@20	[0.106503],	||	 recall@50	[0.188057]
precision@5	[0.263377],	||	 precision@20	[0.207864],	||	 precision@50	[0.155603]

epoch
3
recall@5	[0.034252],	||	 recall@20	[0.106050],	||	 recall@50	[0.189113]
precision@5	[0.262649],	||	 precision@20	[0.207988],	||	 precision@50	[0.155748]

epoch
4
recall@5	[0.035456],	||	 recall@20	[0.105614],	||	 recall@50	[0.189202]
precision@5	[0.264470],	||	 precision@20	[0.207558],	||	 precision@50	[0.155974]

epoch
5
recall@5	[0.034924],	||	 recall@20	[0.106960],	||	 recall@50	[0.191807]
precision@5	[0.262881],	||	 precision@20	[0.208816],	||	 precision@50	[0.158189]

epoch
6
recall@5	[0.035750],

In [84]:
print('recall@5\t[%.6f],\t||\t recall@20\t[%.6f],\t||\t recall@50\t[%.6f]' % (rec_bpr[0], rec_bpr[1], rec_bpr[2]))
print('precision@5\t[%.6f],\t||\t precision@20\t[%.6f],\t||\t precision@50\t[%.6f]' % (pre_bpr[0], pre_bpr[1], pre_bpr[2]))
print('')

recall@5	[0.048788],	||	 recall@20	[0.137591],	||	 recall@50	[0.251897]
precision@5	[0.355629],	||	 precision@20	[0.271573],	||	 precision@50	[0.210166]



**Question (5 points):** do you observe a better result compared with models implemented in previous three parts? What's reason do you think that brings this improvement?

# Part 5. Cool Extension! (10 points)

Just like in the first homework, now is your chance to try an interesting extension. 

### Implement Hybering recommendation

In [81]:
# Load all the recommendations

# non personalised
NP_rec = np_recommendations   #(user id ,50 items ids recommended to the user)
#user user cf
CF_rec = cos_recommend  #(user id ,50 items ids recommended to the user)
#MF
MF_rec = mf_implicit.predict()  #(user id ,50 items ids recommended to the user)
#BPR
BPR_rec = bpr.predict()  #(user id ,50 items ids recommended to the user)

#Load all the precesion and recall for top 50

# precision@k and recall@k values for each recommender
precision_k = np.array([pre_np,   # non-personalized
                      pre_cf,   # collaborative filtering
                      pre_mf,   # matrix factorization
                      pre_bpr])  # BPR

recall_k = np.array([rec_np,      # non-personalized
                   rec_cf,      # collaborative filtering
                   rec_mf,      # matrix factorization
                   rec_bpr])     # BPR


# define weights based on relative performance
w = np.zeros(4)
w[0] = np.mean(precision_k[0]) + np.mean(recall_k[0])  # non-personalized
w[1] = np.mean(precision_k[1]) + np.mean(recall_k[1])  # collaborative filtering
w[2] = np.mean(precision_k[2]) + np.mean(recall_k[2])  # matrix factorization
w[3] = np.mean(precision_k[3]) + np.mean(recall_k[3])  # BPR

w /= np.sum(w)


#find combine above items recommended by each recommender and find top 20 new items

#find score for each item on the basis of above weights and recommend top 50 scored items

#combine all reccommended items for each user
new_items = [[] for u in range(num_users)]

# Combine the recommendations
for u in range(0,num_users):
    combined_list = []
    for lst in [NP_rec[u] , CF_rec[u] , MF_rec[u] , BPR_rec[u]]:
        combined_list.extend(lst)
    combined_list = list(set(combined_list))
    new_items[u] = combined_list

#find score for each movie for a user and add it to recom list
new_hybrid_rec = [[] for u in range(num_users)]


for u in range(0,num_users):
    mv = new_items[u]
    movies = [int(num) for num in mv]
    score_movies = np.zeros(len(movies))
    k = 0
    for m in movies:
        s = 0
        if m in NP_rec[u]:
            s += w[0]
        if m in CF_rec[u]:
            s += w[1]
        if m in MF_rec[u]:
            s += w[2]
        if m in BPR_rec[u]:
            s += w[3]
            
        score_movies[k] = s
        k+=1
    
    score_movies_sorted = np.argsort(score_movies)[::-1]
    recommended_movies = [movies[i] for i in score_movies_sorted[:50]]
    new_hybrid_rec[u] = recommended_movies
        
hybrid_rec  = np.array(new_hybrid_rec)

In [82]:
# lets test this reccommender

# Calculate recall@k and precision@k with k=5, 20, 50 and print out the average over all users for these 9 metrics.
# Your Code Here...
recommendation = hybrid_rec

recalls = np.zeros(3)
precisions = np.zeros(3)
user_count = 0.

user_test_like = []
for u in range(num_user):
    user_test_like.append(np.where(test_mat[u, :] > 0)[0])

for u in range(num_user):
        
    test_like = user_test_like[u]
    test_like_num = len(test_like)
    if test_like_num == 0:
        continue
    rec = recommendation[u, :]
    hits = np.zeros(3)
    for k in range(50):
        if rec[k] in test_like:
            if k < 50:
                hits[2] += 1
                if k < 20:
                    hits[1] += 1
                    if k < 5:
                        hits[0] += 1
    recalls[0] += (hits[0] / test_like_num)
    recalls[1] += (hits[1] / test_like_num)
    recalls[2] += (hits[2] / test_like_num)
    precisions[0] += (hits[0] / 5.)
    precisions[1] += (hits[1] / 20.)
    precisions[2] += (hits[2] / 50.)
    user_count += 1

recalls /= user_count
precisions /= user_count

print('recall@5\t[%.6f],\t||\t recall@20\t[%.6f],\t||\t recall@50\t[%.6f]' % (recalls[0], recalls[1], recalls[2]))
print('precision@5\t[%.6f],\t||\t precision@20\t[%.6f],\t||\t precision@50\t[%.6f]' % (precisions[0], precisions[1], precisions[2]))
print('')


recall@5	[0.058835],	||	 recall@20	[0.168515],	||	 recall@50	[0.289817]
precision@5	[0.370265],	||	 precision@20	[0.300075],	||	 precision@50	[0.227742]



# Collaboration Declarations

** You should fill out your collaboration declarations here.**

**Reminder:** You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Piazza, search StackOverflow, etc. But if you do get help in this way, you must inform us by filling out the Collaboration Declarations at the bottom of this notebook.

Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.